In [108]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
import re

from utilities.preprocessors import preprocess, assign_gender, map_value_to_index
from utilities.visualizers import view_rows
from utilities.loaders import load_file, load_meta_data, load_lookup_array
from models.fil_lstm_glove import load_lstm_model

from dotenv import load_dotenv
from pathlib import Path

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
# load environment variables path
env_dir = Path('../').resolve()
load_dotenv(os.path.join(env_dir, '.env'))

Python-dotenv could not parse statement starting at line 53
Python-dotenv could not parse statement starting at line 57
Python-dotenv could not parse statement starting at line 63
Python-dotenv could not parse statement starting at line 64
Python-dotenv could not parse statement starting at line 66
Python-dotenv could not parse statement starting at line 68
Python-dotenv could not parse statement starting at line 70
Python-dotenv could not parse statement starting at line 74
Python-dotenv could not parse statement starting at line 75
Python-dotenv could not parse statement starting at line 77
Python-dotenv could not parse statement starting at line 83
Python-dotenv could not parse statement starting at line 84
Python-dotenv could not parse statement starting at line 85
Python-dotenv could not parse statement starting at line 86
Python-dotenv could not parse statement starting at line 89
Python-dotenv could not parse statement starting at line 90
Python-dotenv could not parse statement 

True

# load dataframe

In [110]:
conn_info = pd.read_csv('../documents/conn_info.csv', index_col=0)
conn_info

,conn_link,conn_name,email,mobile_no,company_name,gender
0,https://www.linkedin.com/in/mhelbert-paredes-c...,Mhelbert Paredes,mhelbert.paredes17@gmail.com,0,Everise · Full-time,male
1,https://www.linkedin.com/in/mvidanes/,Mari Anthony Vidanes,maricvid@amazon.com,09216720764 / 09913454352 (Mobile),PSG Global Solutions · Full-time,male
2,https://www.linkedin.com/in/krystle-tacpalan-a...,Krystle Tacpalan,krystleannetacpalan@gmail.com,0,"Temps and Staffers, Inc. · Internship",female
3,https://www.linkedin.com/in/venze-visda-chra-a...,Venze Visda,vcvisda@unionbankph.com,09298282188 (Mobile),Christian Dior Couture · Contract,male
4,https://www.linkedin.com/in/jsales365/,Carlito Jr Sales,NaN,+44(0)7537188736 (Mobile),Pyramid Recruitment Ltd · Contract,male
...,...,...,...,...,...,...
1261,https://www.linkedin.com/in/jamie-rose-cleofas...,Jamie Rose Cleofas,NaN,0,transcosmos Asia Philippines Inc. · Full-time,female
1262,https://www.linkedin.com/in/jovit-frias-06a561...,Jovit Frias,jovitfrias@gmail.com,0,RCM Health Care Services · Full-time,male
1263,https://www.linkedin.com/in/danlopezprofessional/,Mark Daniel Lopez,danlopezatwork@gmail.com,0,Abacus Service Corporation · Full-time,male
1264,https://www.linkedin.com/in/daena-michaela-roy...,Daena Michaela Ceria,dmrc_06@yahoo.com,09983696222 (Mobile),RingCentral · Full-time,female


In [111]:
conn_info.dtypes

conn_link       object
conn_name       object
email           object
mobile_no       object
company_name    object
gender          object
dtype: object

In [112]:
conn_info = conn_info.astype('category')
conn_info

,conn_link,conn_name,email,mobile_no,company_name,gender
0,https://www.linkedin.com/in/mhelbert-paredes-c...,Mhelbert Paredes,mhelbert.paredes17@gmail.com,0,Everise · Full-time,male
1,https://www.linkedin.com/in/mvidanes/,Mari Anthony Vidanes,maricvid@amazon.com,09216720764 / 09913454352 (Mobile),PSG Global Solutions · Full-time,male
2,https://www.linkedin.com/in/krystle-tacpalan-a...,Krystle Tacpalan,krystleannetacpalan@gmail.com,0,"Temps and Staffers, Inc. · Internship",female
3,https://www.linkedin.com/in/venze-visda-chra-a...,Venze Visda,vcvisda@unionbankph.com,09298282188 (Mobile),Christian Dior Couture · Contract,male
4,https://www.linkedin.com/in/jsales365/,Carlito Jr Sales,NaN,+44(0)7537188736 (Mobile),Pyramid Recruitment Ltd · Contract,male
...,...,...,...,...,...,...
1261,https://www.linkedin.com/in/jamie-rose-cleofas...,Jamie Rose Cleofas,NaN,0,transcosmos Asia Philippines Inc. · Full-time,female
1262,https://www.linkedin.com/in/jovit-frias-06a561...,Jovit Frias,jovitfrias@gmail.com,0,RCM Health Care Services · Full-time,male
1263,https://www.linkedin.com/in/danlopezprofessional/,Mark Daniel Lopez,danlopezatwork@gmail.com,0,Abacus Service Corporation · Full-time,male
1264,https://www.linkedin.com/in/daena-michaela-roy...,Daena Michaela Ceria,dmrc_06@yahoo.com,09983696222 (Mobile),RingCentral · Full-time,female


In [113]:
conn_info['mobile_no'] == "0"

0        True
1       False
2        True
3       False
4       False
        ...  
1261     True
1262     True
1263     True
1264    False
1265    False
Name: mobile_no, Length: 1266, dtype: bool

In [114]:
conn_info['gender'].dtype

CategoricalDtype(categories=['female', 'male'], ordered=False, categories_dtype=object)

In [115]:
pd.isnull(conn_info['gender'])

0       False
1       False
2       False
3       False
4       False
        ...  
1261    False
1262    False
1263    False
1264    False
1265    False
Name: gender, Length: 1266, dtype: bool

In [116]:
conn_info['gender'].unique()

['male', 'female']
Categories (2, object): ['female', 'male']

In [117]:
conn_info['conn_name'] = conn_info['conn_name'].apply(preprocess)
conn_info.head()

Aaron Mitchel A.
Aaron Mitchel Ancheta
Aaron Penaco
Aaron Santos
Abegail Jean Bañez
Abigail Baliwang
Abigail Digamon
Abigail Inobaya
Abigail Lyle Arquero
Aby Aquino
Adrian Joseph
Adriane Jomel Sodusta
Adriane Valderrama
Aerdan Joseph Lacson
Aerdan Joseph Lacson Clsswb
Agot Reyes
Aimeh Grace Nono Lazaro
Aimeh Nono Lazaro
Aira Batang
Aira Danielle Cacho
Aira Magallon
Airene Ramilo
Airizh Criztal Santos
Aisha Locsin Red
Aishwarya Tanwar
Aiza Ricohermoso
Al Christian Agngarayngay
Al D.
Al John Ezekiel Udtuhan
Al Valencia
Aldrin Astrande
Alecks Alamo
Alecs Rodriguez
Alfred Tolentino
Allan Repalda
Allen Dionio
Allexa Contreras
Alliza Mendoza
Allyssa Reyes
Almira Aquino
Almira Cayabyab
Aloha Ferrer
Aloicius Justine Kochensparger
Altair Deocampo
Althea Espinosa
Alvin Troy Catacutan
Alyanna Monica Coladilla
Alyssa Bela Ong
Alyssa Christa Cunanan
Alyssa Jean Sonza
Alyssa Kirra Macarilay
Alyssa Marie Lansangan
Alyssa Marie Reyes
Alyssa Palamiano
Alyssa Solitario
Ameera V.
Ameera Valdez
Amiel Ayro

,conn_link,conn_name,email,mobile_no,company_name,gender
0,https://www.linkedin.com/in/mhelbert-paredes-c...,Mhelbert Paredes,mhelbert.paredes17@gmail.com,0,Everise · Full-time,male
1,https://www.linkedin.com/in/mvidanes/,Mari Anthony Vidanes,maricvid@amazon.com,09216720764 / 09913454352 (Mobile),PSG Global Solutions · Full-time,male
2,https://www.linkedin.com/in/krystle-tacpalan-a...,Krystle Tacpalan,krystleannetacpalan@gmail.com,0,"Temps and Staffers, Inc. · Internship",female
3,https://www.linkedin.com/in/venze-visda-chra-a...,Venze Visda,vcvisda@unionbankph.com,09298282188 (Mobile),Christian Dior Couture · Contract,male
4,https://www.linkedin.com/in/jsales365/,Carlito Jr Sales,NaN,+44(0)7537188736 (Mobile),Pyramid Recruitment Ltd · Contract,male


In [118]:
view_rows(conn_info, 'conn_name')

Mhelbert Paredes
Mari Anthony Vidanes
Krystle Tacpalan
Venze Visda
Carlito Jr Sales
Brian Onoh
Rommel Echave
Ryan Louie Rivera
Remfel Belandres
Cindy De Leon
April Christine Quicosa
Lyana Torrecampo
Whenly Kaye Bonganay
Louise Claudine Jayme
Almira Cayabyab
Floriedel Sanchez
Christine Joi Lopez
Donahrene Dela Torre
Fatima Allona Bolano
Janna Liwanag
Abigail Inobaya
Annaliza Tuazon
Marlon Martin
Love Marquez Sevilla
Sheana Sunga
John Martin Gabata
Tor Corpuz
Krystel Grace Dimalibot
Paul Alessandrian Legaspi
Judd Rojas
Adriane Jomel Sodusta
Christian Angelo Bautista
Jane Cervantes
Casandra Nicole Javier
Jona Ochea
Elaine Pagaduan
Yale Abungar
Maine Cua
Almira Aquino
Athena Grace Macasinag
Ana Anorico
Vanessa Sefuentes
Jean Andreassen Temblor
Andrea Toledo
Kevin Brylle Santos
Mereden Gamosa
Christine Beatriz Aguilera
Christian Paul Diaz
Chinnie Obispo
Mica Baes
John Daven Torres
Joel Catalan
Ralph Chester Barro
Mariel Publico
Erika Dinginbayan
Khim Gomez
Jessa Joguilon
Ericson Ramos
Mc Ca

In [119]:
conn_info

,conn_link,conn_name,email,mobile_no,company_name,gender
0,https://www.linkedin.com/in/mhelbert-paredes-c...,Mhelbert Paredes,mhelbert.paredes17@gmail.com,0,Everise · Full-time,male
1,https://www.linkedin.com/in/mvidanes/,Mari Anthony Vidanes,maricvid@amazon.com,09216720764 / 09913454352 (Mobile),PSG Global Solutions · Full-time,male
2,https://www.linkedin.com/in/krystle-tacpalan-a...,Krystle Tacpalan,krystleannetacpalan@gmail.com,0,"Temps and Staffers, Inc. · Internship",female
3,https://www.linkedin.com/in/venze-visda-chra-a...,Venze Visda,vcvisda@unionbankph.com,09298282188 (Mobile),Christian Dior Couture · Contract,male
4,https://www.linkedin.com/in/jsales365/,Carlito Jr Sales,NaN,+44(0)7537188736 (Mobile),Pyramid Recruitment Ltd · Contract,male
...,...,...,...,...,...,...
1261,https://www.linkedin.com/in/jamie-rose-cleofas...,Jamie Rose Cleofas,NaN,0,transcosmos Asia Philippines Inc. · Full-time,female
1262,https://www.linkedin.com/in/jovit-frias-06a561...,Jovit Frias,jovitfrias@gmail.com,0,RCM Health Care Services · Full-time,male
1263,https://www.linkedin.com/in/danlopezprofessional/,Mark Daniel Lopez,danlopezatwork@gmail.com,0,Abacus Service Corporation · Full-time,male
1264,https://www.linkedin.com/in/daena-michaela-roy...,Daena Michaela Ceria,dmrc_06@yahoo.com,09983696222 (Mobile),RingCentral · Full-time,female


In [120]:
conn_info.dtypes

conn_link       category
conn_name       category
email           category
mobile_no       category
company_name    category
gender          category
dtype: object

In [121]:
def simplify_comp_name(comp_name):
    new_comp_name = re.sub(r"· (Full-time|Freelance|Contract|Internship|Freelance|Part-time)", "", comp_name)
    new_comp_name = new_comp_name.strip()
    
    return new_comp_name
    
conn_info['company_name'] = conn_info['company_name'].apply(simplify_comp_name)

In [122]:
hyper_params = load_meta_data('./saved/misc/fil_name_gender_lstm_hyper_params.json')
hyper_params

{'input_shape': 21,
 'vocab_len': 29,
 'emb_dim': 256,
 'emb_matrix': None,
 'n_a': 128,
 'rnn_drop_prob': 0.2,
 'dense_drop_prob': 0.2,
 'n_units': 1}

In [123]:
lstm_model = load_lstm_model(**hyper_params)
lstm_model

input shape: 21


<Sequential name=fil-name-gender-lstm, built=True>

In [124]:
lstm_model.summary()

Model: "fil-name-gender-lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding-layer (Embedding)     │ (None, 21, 256)        │         7,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 401,921 (1.53 MB)

 Trainable params: 401,921 (1.53 MB)

 Non-trainable params: 0 (0.00 B)

In [125]:
lstm_model.load_weights('./saved/weights/fil_name_gender_lstm_10_0.8914.weights.keras')

In [126]:
chars = load_lookup_array('./saved/misc/fil_name_gender_lstm_lookup.txt')
chars

['0',
 '8',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [127]:
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [128]:
char_to_idx.get_vocabulary()

['[UNK]',
 np.str_('0'),
 np.str_('8'),
 np.str_('a'),
 np.str_('b'),
 np.str_('c'),
 np.str_('d'),
 np.str_('e'),
 np.str_('f'),
 np.str_('g'),
 np.str_('h'),
 np.str_('i'),
 np.str_('j'),
 np.str_('k'),
 np.str_('l'),
 np.str_('m'),
 np.str_('n'),
 np.str_('o'),
 np.str_('p'),
 np.str_('q'),
 np.str_('r'),
 np.str_('s'),
 np.str_('t'),
 np.str_('u'),
 np.str_('v'),
 np.str_('w'),
 np.str_('x'),
 np.str_('y'),
 np.str_('z')]

# encode names into numerical format using the mapper you used eariler

In [129]:
names = ["mikhail", "leonidas", "maria"]
seqs = [[char_to_idx(char).numpy().tolist() for char in name] for name in names]
seqs_padded = tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=hyper_params["input_shape"], padding='post', truncating='post')
seqs_padded

array([[15, 11, 13, 10,  3, 11, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [14,  7, 17, 16, 11,  6,  3, 21,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [15,  3, 20, 11,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0]], dtype=int32)

In [130]:
probs = lstm_model.predict(seqs_padded)
probs

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step


array([[0.04733736],
       [0.20695764],
       [0.9556533 ]], dtype=float32)

In [131]:
labels = (probs >= 0.5).astype('int').reshape(-1)
labels

array([0, 0, 1])

In [132]:
v_func = np.vectorize(lambda label: "male" if label == 0 else "female")
decoded_labels = v_func(labels)

In [133]:
decoded_labels.tolist()

['male', 'male', 'female']

In [134]:
def predict_gender(row):
     # extract gender if any
    gender = row["gender"]
    name = row["conn_name"]
    print(f"name: {name} {type(name)}")
    print(f"gender: {gender} {type(gender)}", end='\n\n')

    # check if gender is null and name is also present
    # if there is no name and gender is null or "unnull" then move on still
    # because there is no point in having a name and then genderizing it
    if (pd.isnull(gender) or gender == None) and not pd.isnull(name) :

        # if split names is ["first name", "last name"]
        # get the first name by split_names[0] if ["first name 1", ..., "first name n", "last name"]
        # get the first name/s by split_names[]
        name = name.lower()
        split_names = name.split()
        print(f"split names: {split_names}")

        if len(split_names) > 2:
            first_names = split_names[:-1]

            # map first names chars to indeces then pad
            seqs = [[char_to_idx(char).numpy().tolist() for char in name] for name in first_names]
            seqs_padded = tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=hyper_params["input_shape"], padding='post', truncating='post')
            
            # use padded sequences as input to predictive model
            probs = lstm_model.predict(seqs_padded)

            # calculate the mean probability
            mean_prob = np.mean(probs)
            
            # use preidcted probabilities to convert to scalar values 0 and 1
            label = (mean_prob >= 0.5).astype('int')

            decoded_label = "male" if label == 0 else "female"
            
            print(f"decoded label: {decoded_label}")
            return decoded_label
        
        else:
            first_name = split_names[0]

            seqs = [[char_to_idx(char) for char in first_name]]
            seqs_padded = tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=hyper_params["input_shape"], padding='post', truncating='post')
            
            print(seqs_padded)
            # use padded sequences as input to predictive model
            probs = lstm_model.predict(seqs_padded)

            # calculate the mean probability
            mean_prob = np.mean(probs)
            
            # use preidcted probabilities to convert to scalar values 0 and 1
            label = (mean_prob >= 0.5).astype('int')

            decoded_label = "male" if label == 0 else "female"
            
            print(f"decoded label: {decoded_label}")
            return decoded_label

        # response = requests.post(f"https://genderapi.io/api/?key={api_key}&name={name}")
        # headers = response.headers
        # data = response.json()
        # print(data)
        # print(headers)
        # # data = {}

        # # if request gives a request limit reached error
        # # thus returning a json without hte gender key, just
        # # use an alternative return value of the null value 
        # # of the empty cell
        # return data.get('gender', gender)
    else:
        return row['gender']

In [135]:
temp = conn_info.apply(predict_gender, axis=1)

name: Mhelbert Paredes <class 'str'>
gender: male <class 'str'>

name: Mari Anthony Vidanes <class 'str'>
gender: male <class 'str'>

name: Krystle Tacpalan <class 'str'>
gender: female <class 'str'>

name: Venze Visda <class 'str'>
gender: male <class 'str'>

name: Carlito Jr Sales <class 'str'>
gender: male <class 'str'>

name: Brian Onoh <class 'str'>
gender: male <class 'str'>

name: Rommel Echave <class 'str'>
gender: male <class 'str'>

name: Ryan Louie Rivera <class 'str'>
gender: female <class 'str'>

name: Remfel Belandres <class 'str'>
gender: male <class 'str'>

name: Cindy De Leon <class 'str'>
gender: female <class 'str'>

name: April Christine Quicosa <class 'str'>
gender: female <class 'str'>

name: Lyana Torrecampo <class 'str'>
gender: female <class 'str'>

name: Whenly Kaye Bonganay <class 'str'>
gender: female <class 'str'>

name: Louise Claudine Jayme <class 'str'>
gender: female <class 'str'>

name: Almira Cayabyab <class 'str'>
gender: female <class 'str'>

name: 

In [136]:
type(None)

NoneType

In [137]:
temp.unique()

array(['male', 'female'], dtype=object)

In [138]:
temp.astype(bool)

0       True
1       True
2       True
3       True
4       True
        ... 
1261    True
1262    True
1263    True
1264    True
1265    True
Length: 1266, dtype: bool

In [139]:
conn_info['gender'] = temp

In [140]:
conn_info

,conn_link,conn_name,email,mobile_no,company_name,gender
0,https://www.linkedin.com/in/mhelbert-paredes-c...,Mhelbert Paredes,mhelbert.paredes17@gmail.com,0,Everise,male
1,https://www.linkedin.com/in/mvidanes/,Mari Anthony Vidanes,maricvid@amazon.com,09216720764 / 09913454352 (Mobile),PSG Global Solutions,male
2,https://www.linkedin.com/in/krystle-tacpalan-a...,Krystle Tacpalan,krystleannetacpalan@gmail.com,0,"Temps and Staffers, Inc.",female
3,https://www.linkedin.com/in/venze-visda-chra-a...,Venze Visda,vcvisda@unionbankph.com,09298282188 (Mobile),Christian Dior Couture,male
4,https://www.linkedin.com/in/jsales365/,Carlito Jr Sales,NaN,+44(0)7537188736 (Mobile),Pyramid Recruitment Ltd,male
...,...,...,...,...,...,...
1261,https://www.linkedin.com/in/jamie-rose-cleofas...,Jamie Rose Cleofas,NaN,0,transcosmos Asia Philippines Inc.,female
1262,https://www.linkedin.com/in/jovit-frias-06a561...,Jovit Frias,jovitfrias@gmail.com,0,RCM Health Care Services,male
1263,https://www.linkedin.com/in/danlopezprofessional/,Mark Daniel Lopez,danlopezatwork@gmail.com,0,Abacus Service Corporation,male
1264,https://www.linkedin.com/in/daena-michaela-roy...,Daena Michaela Ceria,dmrc_06@yahoo.com,09983696222 (Mobile),RingCentral,female


In [141]:
conn_info.to_csv('../documents/conn_info.csv')